# Вебинар 6. Практика работы с временем и датами

# Проверка связи

Если у вас нет звука:
*   убедитесь, что на вашем устройстве и в колонках включён звук
*   обновите страницу вебинара или закройте страницу и заново присоединитесь к вебинару
*   откройте вебинар в другом браузере
*   перезагрузите ваше устройство и попытайтесь войти снова

Поставьте в чат:
* «+» — если видно и слышно
* «–» — если нет

## О спикере

**Олег Булыгин**
- эксперт по Python и data science;
- занимался аналитикой СМИ, IT-аудитом в финтехе;
- работал в научно-производственных компаниях космической отрасли;
- с 2020 года реализую собственные проекты, занимаюсь IT-образованием и консультированием;
- с 2017 года в IT-образовании, стремлюсь сделать обучение качественным и честным для всех, кто хочет развиваться в IT-сфере.

## Правила участия

*   Приготовьте блокнот и ручку, чтобы записывать важные мысли и идеи
*   Продолжительность вебинара — 80 минут
*   Вы можете писать свои вопросы в чате или задавать их вслух
*   Запись вебинара будет доступна в личном кабинете

# Цели занятия

- Познакомиться с типичными проблемами со временем, ведущими к ошибкам при обработке и интерпретации данных.
- Научиться работать с датами в Python.
- Рассмотреть формат Unixtime и узнать, как с ним работать.
- Научиться работать с ошибками в Python.

# План занятия

1. Парсинг и стандартизация
2. Синхронизация часов
3. Арифметика дат и генерация последовательностей
4. Unix timestamp
5. Контекстная обработка ошибок

## Условие задачи

Представьте, что у нас есть три фрагмента логов из разных систем, все они относятся к одному и тому же промежутку времени.

> **1. Лог с веб-сервера Apache (ОС: Linux):**
> ```
> [02/Oct/2022:15:04:05 +0300] "POST /admin/login.php HTTP/1.1" 401 "Unauthorized"
> ```

> **2. Событие безопасности с контроллера домена (ОС: Windows Server):**
> ```
> 2022-10-02 12:04:08Z, Security, EventID 4625, "An account failed to log on."
> ```

> **3. Выгрузка из SIEM-системы (Splunk/ELK):**
> ```
> 1664712246, "High rate of failed logins detected from IP 8.8.8.8"
> ```

Посмотрите на эти три строчки. На первый взгляд, кажется, что это события из разного времени: `15:04`, `12:04` и большое число `1664712246`.

Системы работают на разных ОС, находятся в разных странах, а разработчики ПО используют десятки различных форматов для записи времени. Важно научиться превращать этот текстовый хаос в структурированные данные.

## 1. Парсинг и cтандартизация

Есть три строки, но для Python — это просто набор символов, не имеющий временного смысла.

**Задача:** научить программу видеть в этих строках дату и время.

Используем для решения этой задачи метод `strptime` из модуля `datetime`(**str**ing **p**arse **time** — разобрать время из строки).

Сфокусируемся на первой строке:

```python
apache_log_string = '[02/Oct/2022:15:04:05 +0300]'
```

Чтобы `strptime` сработал, нужно предоставить ему два аргумента:
1.  Строку, которую нужно разобрать.
2.  **Шаблон (формат)**, который описывает, где в этой строке находятся год, месяц, день и так далее.

*   `02` — это день месяца → `%d`;
*   `Oct` — это сокращенное название месяца → `%b`;
*   `2022` — это год из 4-х цифр → `%Y`;
*   `15` — это час в 24-часовом формате → `%H`;
*   `04` — это минута → `%M`;
*   `05` — это секунда → `%S`;
*   `+0300` — это часовой пояс, смещение относительно UTC → `%z`.

Все остальные символы (`[`, `/`, `:`, ` `, `]`) — это литералы, которые необходимо вставить в строку формата на те же места.

Собираем все вместе:

In [ ]:
# Импортируем класс datetime из стандартного модуля datetime
from datetime import datetime

# Строка
apache_log_string = '[02/Oct/2022:15:04:05 +0300]'

# Дешифратор
apache_format_template = '[%d/%b/%Y:%H:%M:%S %z]'

# Выполняем парсинг
event_time_apache = datetime.strptime(apache_log_string, apache_format_template)

# Смотрим, что получилось
print(f"Тип полученного объекта: {type(event_time_apache)}")
print(f"Значение объекта: {event_time_apache}")

# Обратите внимание на свойство tzinfo (timezone info)
print(f"Информация о часовом поясе: {event_time_apache.tzinfo}")

Посмотрите на конец вывода: `+03:00`. Благодаря директиве `%z`, Python не просто узнал время, он узнал его **контекст**.

Типы объектов:

*   **наивные (naive) объекты `datetime`** — не имеют информации о часовом поясе (например, `15:04:05`: неизвестно это время в Москве, Лондоне или Токио).
*   **осведомлённые (aware) объекты `datetime`** — содержат информацию о часовом поясе (например, объект `event_time_apache` именно такой: точно известно, что `15:04:05` во временной зоне со смещением +3 часа от UTC).

### **Формат ISO 8601**

Стандартный формат **ISO 8601** выглядит так: `YYYY-MM-DDTHH:MM:SS.ffffff+HH:MM`.

Например: `2022-10-02T12:04:08+00:00`.

Это предпочтительный формат для логов, API и любого межмашинного взаимодействия, потому что он однозначен и легко читается.

В Python для него есть свой, более удобный метод — `fromisoformat()`.

Посмотрим на вторую строку, лог с Windows Server:
`2022-10-02 12:04:08Z, Security, EventID 4625, "An account failed to log on."`

Часть с датой `2022-10-02 12:04:08Z` почти соответствует ISO 8601. Буква `Z` ("Zulu") — это обозначение для UTC (`+00:00`). `fromisoformat` не понимает `Z` напрямую, но мы можем легко это исправить.

In [ ]:
windows_log_string = '2022-10-02 12:04:08Z'

# Подготовим строку для парсинга: заменим ' ' на 'T' и 'Z' на смещение UTC
iso_compatible_string = windows_log_string.replace(' ', 'T').replace('Z', '+00:00')

print(f"Подготовленная строка: {iso_compatible_string}")

# Используем метод fromisoformat
event_time_windows = datetime.fromisoformat(iso_compatible_string)

# Смотрим результат
print(f"Тип полученного объекта: {type(event_time_windows)}")
print(f"Значение объекта: {event_time_windows}")
print(f"Информация о часовом поясе: {event_time_windows.tzinfo}")

### Библиотека `dateutil`

Сторонняя библиотека **`dateutil`** — инструмент для нестандартных случаев, когда составление шаблона `strptime` может потребовать много времени и трудозатрат.

Её парсер может угадать формат самых разных строк без явного указания шаблона.


In [ ]:
from dateutil.parser import parse

# Строка в человекочитаемом формате
human_readable_string = "Thursday, 18 August 1977 10:30 PM"


In [ ]:
event_time_human = parse(human_readable_string)
print(f"Результат парсинга: {event_time_human}")
print(f"Тип объекта: {type(event_time_human)}")
print(f"Информация о часовом поясе: {event_time_human.tzinfo}")

`parse` справился, но обратите внимание на `tzinfo=None`. `dateutil` создал **"наивный"** объект, так как в исходной строке не было информации о часовом поясе. Это его слабое место и потенциальная ловушка.

**Плюсы `dateutil`:**
*   гибкость — понимает десятки форматов из коробки;
*   экономит время, когда формат неизвестен или сложен.

**Минусы:**
*   медленнее, чем `strptime` или `fromisoformat`, что критично при обработке большого количества записей;
*   может ошибиться на неоднозначных форматах (например, `01-02-03` — это 1 февраля или 2 января).



## Ваши вопросы

## 2. Синхронизация часов

Мы успешно преобразовали строки в объекты `datetime`:

*   `event_time_apache`: `2022-10-02 15:04:05+03:00`
*   `event_time_windows`: `2022-10-02 12:04:08+00:00`

**Задача:** узнать, какое из этих событий произошло раньше.

Интуитивно можно предположить, что `15:04` в Москве — это `12:04` в UTC, но нужен программный, абсолютно надежный способ сравнения.

Наглядно продемонстрируем, почему нельзя смешивать данные.

Например: есть наивное время из лога системы, которая не записывает часовой пояс, но мы знаем, что она находится в Токио (UTC+9).


In [ ]:
from datetime import datetime

# Время события на Windows сервере в UTC
aware_utc_time = event_time_windows
print(f"Осведомленное время (Windows, UTC): {aware_utc_time}")


# Наивное время из лога, снятого в Токио (UTC+9)
# 21:04:15 по токийскому времени — то же, что 12:04:15 UTC
naive_tokyo_time = datetime(2022, 10, 2, 21, 4, 15)
print(f"Наивное время (Токио):             {naive_tokyo_time}")



if aware_utc_time < naive_tokyo_time:
    print("Событие в UTC произошло раньше, чем в Токио.")
else:
    print("Событие в Токио произошло раньше или одновременно с UTC.")


`TypeError: can't compare offset-aware and offset-naive datetimes`. Python сам защищает нас от этой грубой ошибки. Результат сравнения осведомлённого и наивного времени будет бессмысленным.

Перед любыми сравнениями, вычислениями или построением хронологии все временные метки должны быть приведены к единому формату осведомлённых объектов в едином часовом поясе.

По всеобщему соглашению в IT единым часовым поясом является **UTC (coordinated universal time)**.

### Модуль `zoneinfo`

**Модуль `zoneinfo` **— встроенный инструмент Python (с версии Python 3.9) — использует официальную базу данных часовых поясов IANA (Olson database), которая содержит всю информацию о смещениях, переходах на летнее или зимнее время и исторических изменениях для сотен регионов мира. Это самый надежный и правильный способ работать с часовыми поясами в современном Python.

In [ ]:
# Импортируем ZoneInfo из стандартной библиотеки zoneinfo
from zoneinfo import ZoneInfo

# Создаём объекты для ключевых часовых поясов
# Используем стандартные названия из базы IANA
utc_zone = ZoneInfo("UTC")
moscow_zone = ZoneInfo("Europe/Moscow")
new_york_zone = ZoneInfo("America/New_York")

print(f"Объект для UTC: {utc_zone}")
print(f"Объект для Москвы: {moscow_zone}")
print(f"Объект для Нью-Йорка: {new_york_zone}")

In [ ]:
# Выводим исходные данные для наглядности
print(f"ИСХОДНОЕ ВРЕМЯ (Apache):  {event_time_apache}")
print(f"ИСХООДНОЕ ВРЕМЯ (Windows): {event_time_windows}")
print("-" * 40)

# Конвертируем время обоих событий в UTC с помощью метода .astimezone()
# Метод принимает в качестве аргумента объект ZoneInfo
event_time_apache_utc = event_time_apache.astimezone(utc_zone)

# Время Windows уже в UTC, но для унификации рабочего процесса и идемпотентности
# кода (повторный вызов не меняет результат) мы всё равно применяем конвертацию.
# Это хорошая практика.
event_time_windows_utc = event_time_windows.astimezone(utc_zone)

# Смотрим на результат
print(f"ВРЕМЯ В UTC (Apache):     {event_time_apache_utc}")
print(f"ВРЕМЯ В UTC (Windows):    {event_time_windows_utc}")

# Теперь сравнение корректно и безопасно
if event_time_apache_utc < event_time_windows_utc:
    print("\nСобытие на веб-сервере Apache произошло раньше.")
else:
    print("\nСобытие на контроллере домена Windows произошло раньше.")

Результат:
*   событие на веб-сервере произошло в `12:04:05 UTC`;
*   событие на контроллере домена — в `12:04:08 UTC`.

Соответственно, сначала произошла неуспешная авторизация на веб-сервере, через 3 секунды после этого — неуспешная попытка входа на контроллере домена.



### **Создание осведомлённых объектов с нуля**


**Пример:** мы получили наивный объект (например, от `dateutil.parser`), но из контекста (из имени файла или IP-адреса) знаем его часовой пояс.

Использовать `.astimezone()` нельзя, потому что он работает только с осведомлёнными объектами.

Необхоимо сначала прикрепить к объекту информацию о часовом поясе. Для этого используется метод `.replace()`.


In [ ]:
# Представим, что есть наивная временная метка
naive_datetime = datetime(2023, 10, 26, 14, 30, 0)
print(f"Наивное время: {naive_datetime}")

# Мы знаем, что это время Калининграда (UTC+2)
kaliningrad_zone = ZoneInfo("Europe/Kaliningrad")

# Прикрепляем часовой пояс
aware_datetime = naive_datetime.replace(tzinfo=kaliningrad_zone)
print(f"Осведомлённое время: {aware_datetime}")

# И теперь можем легко перевести его в UTC для анализа
aware_datetime_utc = aware_datetime.astimezone(utc_zone)
print(f"Время в UTC: {aware_datetime_utc}")


Двухэтапный процесс **(`.replace()` -> `.astimezone()`)** — стандартный и безопасный паттерн для работы с данными, у которых часовой пояс изначально отсутствовал, но был известен из внешнего контекста.

## Ваши вопросы

## 3. Арифметика дат и генерация последовательностей

Основной инструмент для арифметики дат — класс `timedelta`. `timedelta` — это не конкретное время, а **длительность**, интервал (3 дня, 5 часов, 10 минут и 30 секунд и так далее).

Посмотрим, как работает `timedelta`.

In [ ]:
from datetime import timedelta

# Ключевые временные метки в UTC
print(f"Apache Event (UTC): {event_time_apache_utc}")
print(f"Windows Event (UTC): {event_time_windows_utc}")

# 1. Вычислим точную разницу между событиями
time_difference = event_time_windows_utc - event_time_apache_utc
print(f"\nРазница между событиями: {time_difference}")
print(f"Тип объекта разницы: {type(time_difference)}")

Результат вычитания двух объектов `datetime` — это объект `timedelta`. Можно точно определить, что события разделяют 3 секунды.

**Представим:** согласно внутренним регламентам, после обнаружения события, похожего на брутфорс, необходимо проанализировать всю активность за следующие 5 минут.

**Задача: **вычислить конец этого интервала.

In [ ]:
# Определяем интервал в 5 минут
analysis_window = timedelta(minutes=5)

# Начало окна — время первого события
start_window = event_time_apache_utc

# Конец окна
end_window = start_window + analysis_window

print(f"\nАнализ логов необходимо провести в интервале:")
print(f"С: {start_window}")
print(f"ПО: {end_window}")

`timedelta` поддерживает множество аргументов: `days`, `hours`, `minutes`, `seconds`, `microseconds`, `weeks`. Можно использовать и отрицательные значения для вычитания.

In [ ]:
# Пример: найти время за 1 час до инцидента
pre_incident_time = start_window - timedelta(hours=1)
print(f"\nВремя за час до инцидента: {pre_incident_time}")

### **Подготовка данных для отчёта с помощью метода `strftime`**

Чтобы преобразовать данные в удобный  для человека формат необходимо выполнить форматирование — процесс, обратный парсингу.

Метод `strftime` (**str**ing **f**ormat **time**) позволяет превратить объект `datetime` в строку любого желаемого формата, используя те же директивы, что `strptime`.

In [ ]:
# Время первого события в UTC
event_time = event_time_apache_utc

# Формат для отчёта: "02 Oct 2022, 12:04:05 PM UTC"
report_format = "%d %b %Y, %I:%M:%S %p %Z" # %I — 12-часовой формат, %p — AM/PM, %Z — имя часового пояса

# Создаём строку для отчёта
report_string = event_time.strftime(report_format)

print(f"Исходный объект: {event_time}")
print(f"Строка для отчёта: {report_string}")

# Другой пример, более машинный формат
log_format = "event_time=%Y%m%d-%H%M%S"
print(f"Строка для логов: {event_time.strftime(log_format)}")

## Ваши вопросы

## 4. Unix timestamp

Разберём последнюю строку

> **`1664712246`, "High rate of failed logins detected from IP 8.8.8.8"**

Число `1664712246` это — **Unix timestamp** (epoch time, метка времени).

**Unix timestamp** — способ представления момента времени в виде целого числа, которое обозначает количество секунд, прошедших с полуночи 1 января 1970 года по UTC.

Точка отсчёта `1970-01-01 00:00:00 UTC` называется эпохой Unix.

Преимущества Unix timestamp:

1.  **Однозначность.** Не использует часовые пояса, поэтому нет риска неправильной интерпретации. Всегда UTC.
2.  **Эффективность.** Хранить одно число (integer или float) гораздо выгоднее по памяти и скорости обработки, чем строку.
3.  **Универсальность.** Почти все языки программирования и системы умеют работать с этим форматом по умолчанию.

Переведём строку в человекочитаемый формат. Для этого в Python используется метод `datetime.fromtimestamp()`.

**Важно:** `fromtimestamp()` по умолчанию создаёт наивный объект `datetime` в локальном часовом поясе устройства, поэтому необходимо явно указывать часовой пояс UTC.

In [ ]:
# Импортируем timezone из модуля datetime для доступа к UTC
from datetime import datetime, timezone

siem_timestamp = 1664712246

# Преобразуем timestamp в осведомлённый datetime в UTC
event_time_siem_utc = datetime.fromtimestamp(siem_timestamp, tz=timezone.utc)

print(f"Unix Timestamp: {siem_timestamp}")
print(f"Время события SIEM (UTC): {event_time_siem_utc}")

**Результат:** у нас есть время третьего события сразу в нужном формате UTC.

Выведем все три временные метки, которые мы расшифровали, и отсортируем их.


In [ ]:
# Создаём список всех событий
timeline = [
    (event_time_apache_utc, "Веб-сервер: Неудачная попытка входа в /admin/login.php"),
    (event_time_windows_utc, "Контроллер домена: Событие 4625 — отказ в аутентификации"),
    (event_time_siem_utc, "SIEM: Обнаружена высокая частота неудачных входов")
]

# Сортируем список по времени (первому элементу кортежа)
timeline.sort()

for event_time, description in timeline:
    # Используем strftime для красивого вывода
    print(f"[{event_time.strftime('%Y-%m-%d %H:%M:%S')}] - {description}")

### **Обратное преобразование из `datetime` в Unix timestamp**

Иногда требуется выполнить обратную операцию преобраазования.
Пример: чтобы отправить запрос к API, который требует время в формате Unix.

Для обратного преобразования используется метод `.timestamp()`.

In [ ]:
# Возьмём время первого события
some_datetime_object = event_time_apache_utc

# Получим его Unix timestamp
# Метод .timestamp() у осведомлённых объектов всегда возвращает корректное значение UTC
unix_ts = some_datetime_object.timestamp()

print(f"\nОбъект datetime: {some_datetime_object}")
print(f"Его Unix Timestamp: {unix_ts}")

## Ваши вопросы

## 5. Контекстная обработка ошибок

Что произойдет, если в середине лог-файла на 10 Гб парсеру попадётся битая строка?

```
[02/Oct/2022:15:05:-- INVALID DATA --] "GET /index.html HTTP/1.1" 200
```
Или строка из другого сервиса, которая случайно попала в этот же файл?

Или просто информационное сообщение от системы?

```
Application restart sequence initiated...
```

Парсер, ожидающий шаблон `[%d/%b/%Y:%H:%M:%S %z]`, столкнётся с `INVALID DATA` и аварийно завершит работу с ошибкой `ValueError`. Анализ остановится.

Код должен быть **отказоустойчивым**: уметь обрабатывать ошибки, регистрировать их для последующего анализа и продолжать работу, несмотря ни на что.

Для этого в Python существует конструкция **`try...except`**.

</br>

**Идея:**

Пробуем (`try`) выполнить потенциально опасную операцию (в нашем случае, парсинг даты):

- если операция завершается успешно, работаем с результатом;

- если операция вызывает ошибку (`ValueError`), не даём программе упасть, а перехватываем эту ошибку блоком `except` и выполняем альтернативный код (например, записываем проблемную строку в отдельный файл).

</br>

Напишем функцию, реализующую этот паттерн. Это практически готовый шаблон для использования в ваших рабочих скриптах.


In [ ]:
# Необработанный лог-файл в виде списка строк
# Намеренно добавили в него грязные данные
raw_logs = [
    '[02/Oct/2022:15:04:05 +0300] "POST /admin/login.php HTTP/1.1" 401',
    '[02/Oct/2022:15:04:15 +0300] "POST /admin/login.php HTTP/1.1" 401',
    '--- Kernel panic: unable to mount root fs on unknown-block(0,0) ---', # Системное сообщение
    '[02/Oct/2022:15:04:25 +0300] "POST /admin/login.php HTTP/1.1" 401',
    '[02/Oct/2022:15:04:-- CORRUPTED --] "POST /admin/login.php HTTP/1.1" 401', # Строка с битой датой
    '[02/Oct/2022:15:04:55 +0300] "POST /admin/login.php HTTP/1.1" 200 OK', # Успешный вход
]

# Шаблон для парсинга, который мы уже знаем
apache_format_template = '[%d/%b/%Y:%H:%M:%S %z]'


def robust_log_parser(log_data, format_template):
    """
    Отказоустойчиво парсит список строк лога.

    :param log_data: Список строк для анализа.
    :param format_template: Шаблон для datetime.strptime.
    :return: Кортеж из двух списков: (успешно разобранные, строки с ошибками).
    """
    parsed_events = []
    failed_lines = []

    # Итерируемся по каждой строке в логе
    for i, line in enumerate(log_data):
        try:
            # --- Блок TRY ---
            # Пытаемся выполнить опасную операцию — парсинг
            event_time = datetime.strptime(line, format_template)
            # Если успешно, сохраняем результат
            parsed_events.append({'timestamp': event_time, 'raw_log': line})

        except ValueError:
            # --- Блок EXCEPT ---
            # Если strptime не справился и вызвал ValueError,
            # ловим ошибку и выполняем этот блок кода.
            # Не падаем, а просто фиксируем проблему.
            failed_lines.append({'line_number': i + 1, 'raw_log': line})

    return parsed_events, failed_lines


# Запускаем наш парсер
successful_events, error_lines = robust_log_parser(raw_logs, apache_format_template)

# --- Выводим результаты ---
print("✅ Анализ завершён.")
print(f"   Успешно обработано событий: {len(successful_events)}")
print(f"   Обнаружено строк с ошибками: {len(error_lines)}\n")

print("--- 🔬 Успешно обработанные события ---")
for event in successful_events:
    # Приведём к UTC для единообразия
    ts_utc = event['timestamp'].astimezone(timezone.utc)
    print(f"[{ts_utc.strftime('%H:%M:%S')}] - {event['raw_log']}")

print("\n--- ⚠️ Проблемные строки для ручного анализа ---")
for error in error_lines:
    print(f"Строка #{error['line_number']}: {error['raw_log']}")

**Результат: **

Скрипт не упал, а аккуратно прошёл по всему списку:
*   успешно обработанные строки превратил в объекты `datetime` и сложил в список `parsed_events`;
*   строки, которые не подошли под шаблон, вызвали ошибку `ValueError`, но её перехватил блок `except`, а проблемная строка быладобавлена в список `failed_lines` для последующего ручного разбора;
*   работа была продолжена: найдена строка с успешным входом (`200 OK`), которая шла после битой записи.

Вы можете создавать более сложные конструкции, обрабатывая разные типы ошибок (`IndexError`, `KeyError` и т.д.).


## Ваши вопросы

# Итоги

- Познакомились с типичными проблемами со временем, ведущими к ошибкам при обработке и интерпретации данных.
- Научились работать с датами в Python.
- Рассмотрели формат Unixtime и узнать, как с ним работать.
- Научились работать с ошибками в Python.

📱 Делюсь полезными материалами о Python:
- [Telegram](https://t.me/pythontalk)